In [2]:
import gym
import numpy as np

In [3]:
env = gym.make('FrozenLake-v0')
# on policy agent : constantly updates how it behaves 

[2017-12-22 00:08:41,301] Making new env: FrozenLake-v0


In [4]:
n_state = env.observation_space.n
n_actions = env.action_space.n

In [5]:
print(n_state)
print(n_actions)

16
4


In [6]:
env.render()


SFFF
FHFH
FFFH
HFFG


In [13]:
# agent is not very depentable because 
# it is more depentdant on how luckily agent choose randome correct rewards 
# rather than depending upon learning actual policy
# so need to choose more appropriate epsilon greedy policy

epsilon = 1 # using epsilon greedy strategy 
gamma = 0.95

episodes = 5000
Q = np.zeros([n_state, n_actions])
rewardTracker = [] 

alpha = 0.618 #Learning rate
G=0

for episode in range(1,episodes+1):
    done = False
    G, reward = 0, 0
    state = env.reset()
    firstState = state
    
    while done != True:
        
        if np.random.rand() > epsilon: # using epsilon greedy strategy for explore exploit dilemma
            action = np.argmax(Q[state])
        else:
            action = env.action_space.sample()
            epsilon -= 10**-3
   
        state2, reward, done, info = env.step(action)
        
        Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.argmax(Q[state2]- Q[state, action]))        
        G += reward
        state = state2

    rewardTracker.append(G)

if (sum(rewardTracker[episode-100:episode])/100.0) > 0.78:
    print('--------------------------------------------')
    print('solved after {} episodes with avg return of {}'.format(episodeNum-100, sum(rewardTracker[episodeNum])))

print("Average return of {}".format(sum(rewardTracker)/len(rewardTracker)))

Average return of 0.0002


In [11]:
# best way to solve above problme of dependability 
# is by adding random decaying noise to argmax policy
# each time agent takes an action  

In [18]:
def e_greedy (eps, Q, state, episode):
    
    if np.random.rand() > eps: # using epsilon greedy strategy for explore exploit dilemma
        action = np.argmax(Q[state] + np.random.randn(1,n_actions)/(episode/4))
    else:
        action = env.action_space.sample()
        eps -= 10**-5
            
    return action, eps

In [34]:
def learn_Q(alpha, gamma, eps, numTrainingEpisodes, numTrainingSteps):

    global Q_star
    Q = np.zeros([n_state, n_actions])
    rewardTracker = [] 

    for episode in range(1,numTrainingEpisodes+1):
        G=0
        state = env.reset()
        done = False
        firstState = state

        for step in range(1,numTrainingSteps+1):
            action, eps = e_greedy(eps, Q, state, episode)
            
            state2, reward, done, info = env.step(action)

            Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.argmax(Q[state2]- Q[state, action]))        
            G += reward
            state = state2

        rewardTracker.append(G)

        if episode%(numTrainingEpisodes*.10) == 0 and episode != 0:
            print('Alpha {}, Gamma {}, Epsilon{:04.3f} Episode{} of {}'.format(alpha, gamma, eps, episode, numTrainingEpisodes))
            print("Average total return: {}".format(sum(rewardTracker)/episode))
            print('####-------------------------------------------')
            
        if (sum(rewardTracker[episode-100:episode])/100.0) > 0.78:
            print('--------------------------------------------')
            print('solved after {} episodes with avg return of {}'.format(episodeNum-100, sum(rewardTracker[episode-100:episode])))
            Q_star = Q
            break
        
    Q_star = Q

In [42]:
learn_Q(0.5, 0.99, 0.1, 10000, 300)

Alpha 0.5, Gamma 0.99, Epsilon0.005 Episode1000 of 10000
Average total return: 0.064
####-------------------------------------------
Alpha 0.5, Gamma 0.99, Epsilon0.000 Episode2000 of 10000
Average total return: 0.062
####-------------------------------------------
Alpha 0.5, Gamma 0.99, Epsilon0.000 Episode3000 of 10000
Average total return: 0.063
####-------------------------------------------
Alpha 0.5, Gamma 0.99, Epsilon0.000 Episode4000 of 10000
Average total return: 0.0625
####-------------------------------------------
Alpha 0.5, Gamma 0.99, Epsilon0.000 Episode5000 of 10000
Average total return: 0.0632
####-------------------------------------------
Alpha 0.5, Gamma 0.99, Epsilon0.000 Episode6000 of 10000
Average total return: 0.06266666666666666
####-------------------------------------------
Alpha 0.5, Gamma 0.99, Epsilon0.000 Episode7000 of 10000
Average total return: 0.064
####-------------------------------------------
Alpha 0.5, Gamma 0.99, Epsilon0.000 Episode8000 of 10

In [51]:
def evaluate (Q, numTrainingEpisodes, numTrainingSteps, render):
    rewardTracker = []
    
    for episode in range(1,numTrainingEpisodes+1):
        G=0
        state = env.reset()
  
        for step in range(1,numTrainingSteps+1):
            action = np.argmax(Q[state])
            
            state2, reward, done, info = env.step(action)
            state = state2
            G += reward

            if render == True:
                env.render()
            if done == True:
                break
                
        rewardTracker.append(G)
        
        if episode%(numTrainingEpisodes*.10) == 0 and episode != 0:
            print("Average total return after {} episodes: {:4.3f}".format(episode, sum(rewardTracker)/episode))

In [52]:
evaluate(Q_star, 1, 1000, True)

  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
